In [50]:
import pandas as pd
import csv
import sys
import ast
import random
import numpy as np
from tqdm.notebook import tqdm
sys.path.append('../src/')
from functions import from_tsv_to_list
from functions import save_list_to_tsv_file
sys.path.append("../../")
from image_size import get_image_size  # source: https://github.com/scardine/image_size

In [51]:
# images
image_dir = '../../scraped_photos_final/'

# test
try:
    test_df = pd.read_csv('../model_output/test_model_output.csv', index_col=0)
    in_test = from_tsv_to_list('../../news-navigator/test-A/in.tsv')
except:
    print('(1) No such file or directory')
# val
try:
    val_df = pd.read_csv('../model_output/val_model_output.csv', index_col=0)
    in_val = from_tsv_to_list('../../news-navigator/dev-0/in.tsv')
except:
    print('(2) No such file or directory')
# train
try:
    train_df = pd.read_csv('../model_output/train_model_output.csv', index_col=0)
    in_train = from_tsv_to_list('../../news-navigator/train/in.tsv')
except:
    print('(3) No such file or directory')


(3) No such file or directory


In [52]:
def parse_model_outcome(model_outcome_df, in_file, image_directory):
    # getting images size before rescale
    img_old_sizes_list = []
    for i in range(len(in_file)):
        img_width, img_height = get_image_size.get_image_size(
            image_directory + in_file[i]
        )
        img_old_sizes_list.append([img_width, img_height])
    
    model_outcome_df['old_image_size'] = img_old_sizes_list

    scaler_width, scaler_height = [], []
    for i in range(len(model_outcome_df)):
        old_image_size_width = model_outcome_df['old_image_size'][i][0]
        old_image_size_height = model_outcome_df['old_image_size'][i][1]
        new_image_size_width = np.float(ast.literal_eval(model_outcome_df['new_image_size'][i])[0][0])
        new_image_size_height = np.float(ast.literal_eval(model_outcome_df['new_image_size'][i])[0][0])

        scaler_width.append(np.float(old_image_size_width)/np.float(new_image_size_width))
        scaler_height.append(np.float(old_image_size_height)/np.float(new_image_size_height))

    out_list = []
    for i in range(len(model_outcome_df)):
        pred_labels = ast.literal_eval(model_outcome_df['predicted_labels'][i])
        pred_boxes = ast.literal_eval(model_outcome_df['predicted_boxes'][i])
        out_str = ''
        for ii in range(len(pred_labels)):        
            if int(pred_labels[ii]) == 0:
                label = 'photograph'
            elif int(pred_labels[ii]) == 1:
                label = 'illustration'
            elif int(pred_labels[ii]) == 2:
                label = 'map'
            elif int(pred_labels[ii]) == 3:
                label = 'cartoon'
            elif int(pred_labels[ii]) == 4:
                label = 'editorial_cartoon'
            elif int(pred_labels[ii]) == 5:
                label = 'headline'
            elif int(pred_labels[ii]) == 6:
                label = 'advertisement'
            x0 = str(int(round(pred_boxes[ii][0],0)*scaler_width[i]))
            y0 = str(int(round(pred_boxes[ii][1],0)*scaler_height[i]))
            x1 = str(int(round(pred_boxes[ii][2],0)*scaler_width[i]))
            y1 = str(int(round(pred_boxes[ii][3],0)*scaler_height[i]))

            out_str = out_str + f'{label}:{x0},{y0},{x1},{y1} '
        
        out_str = out_str.strip(" ")
        out_list.append(out_str)

    return out_list



In [53]:
# test
try:
    out_list_test = parse_model_outcome(test_df, in_test, image_dir)
    save_list_to_tsv_file('../../news-navigator/test-A/out.tsv', out_list_test)
except:
    print('(1) No such file or directory')
# val
try:
    out_list_val = parse_model_outcome(val_df, in_val, image_dir)
    save_list_to_tsv_file('../../news-navigator/dev-0/out.tsv', out_list_val)
except:
    print('(2) No such file or directory')
# train
try:
    out_list_train = parse_model_outcome(train_df, in_train, image_dir)
    save_list_to_tsv_file('../../news-navigator/train/out.tsv', out_list_train)
except:
    print('(3) No such file or directory')

(3) No such file or directory
